### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# 使用 colab 環境的同學請執行以下程式碼
%tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
import tensorflow as tf
print(tf.__version__)

import os
from google.colab import drive 
drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
%cd 'gdrive/My Drive/1st-DL-CVMarathon/Day043'
# os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
# %cd cupoy_cv_part4 # 可以自己改路徑

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
[Errno 2] No such file or directory: 'gdrive/My Drive/GEODAC/1st-DL-Marathon/Day043'
/content


In [26]:
# 如果要使用 kaggle API 下載資料的話，請用以下程式碼
os.environ['KAGGLE_USERNAME'] = 'jianjhihlai' # 請按照 Day42 簡報內容，獲取 kaggle api 需要用到的 username
os.environ['KAGGLE_KEY'] = '______' # 請按照 Day42 簡報獲取 kaggle api 需要用到的 key
!kaggle competitions download -c facial-keypoints-detection
!unzip test.zip
!unzip training.zip

  0% 0.00/201k [00:00<?, ?B/s]
100% 201k/201k [00:00<00:00, 25.7MB/s]
 31% 5.00M/16.0M [00:00<00:00, 19.2MB/s]
100% 16.0M/16.0M [00:00<00:00, 45.8MB/s]
 82% 49.0M/60.1M [00:00<00:00, 36.9MB/s]
100% 60.1M/60.1M [00:00<00:00, 75.9MB/s]
  0% 0.00/843k [00:00<?, ?B/s]
100% 843k/843k [00:00<00:00, 119MB/s]
Archive:  test.zip
  inflating: test.csv                
Archive:  training.zip
  inflating: training.csv            


In [0]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [28]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Concatenate, Lambda
from keras import backend as K
from keras import regularizers
from keras import initializers

In [0]:
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name,
            kernel_regularizer=regularizers.l2(0.00004),
            kernel_initializer=initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None))(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x

In [0]:
def inception_resnet_block(x, scale, block_type, activation='relu'):
    '''scale: scaling factor to scale the residuals (i.e., the output of
            passing `x` through an inception module) before adding them
            to the shortcut branch. Let `r` be the output from the residual branch,
            the output of this block will be `x + scale * r`.(簡單來說就是控制Residual branch的比例)'''
    if block_type == 'Incpetion_Block-A':
        branch_0 = Conv2d_bn(x, 32, (1, 1), name='inceptiona_bn_0')
        branch_1 = Conv2d_bn(x, 32, (1, 1), name='inceptiona_bn_1a')
        branch_1 = Conv2d_bn(branch_1, 32, (3, 3), name='Inceptiona_bn_1b')
        branch_2 = Conv2d_bn(x, 32, (1, 1), name='inceptiona_bn_2a')
        branch_2 = Conv2d_bn(branch_2, 48, (3, 3), name='inceptiona_bn_2b')
        branch_2 = Conv2d_bn(branch_2, 64, (3, 3), name='inceptiona_bn_2c')
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Incpetion_Block-B':
        branch_0 = Conv2d_bn(x, 192, (1, 1), name='inceptionb_bn_0')
        branch_1 = Conv2d_bn(x, 128, (1, 1), name='inceptionb_bn_1a')
        branch_1 = Conv2d_bn(branch_1, 160, (1, 7), name='inceptionb_bn_1b')
        branch_1 = Conv2d_bn(branch_1, 192, (7, 1), name='inceptionb_bn_1c')
        branches = [branch_0, branch_1]
    elif block_type == 'Incpetion_Block-C':
        branch_0 = Conv2d_bn(x, 192, (1, 1), name='inceptionc_bn_0')
        branch_1 = Conv2d_bn(x, 192, (1, 1), name='inceptionc_bn_1a')
        branch_1 = Conv2d_bn(branch_1, 192, (1, 3), name='inceptionc_bn_1b')
        branch_1 = Conv2d_bn(branch_1, 192, (3, 1), name='inceptionc_bn_1c')
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "block35", "block17" or "block8", '
                         'but got: ' + str(block_type))
    mixed = Concatenate(axis=3)(branches)
    
    '''確保輸入跟輸出深度相同'''
    up = Conv2d_bn(mixed,K.int_shape(x)[3],1,activation=None)
    
    '''導入殘差結構，並給予權重'''
    
    x = Lambda(lambda inputs, scale: inputs[0]+ inputs[1] * scale, ##提示inputs[0]、inputs[1]
               output_shape=K.int_shape(x)[1:],
               arguments={'scale': scale},)([x,up])
    
    if activation is not None:
        x = Activation(activation)(x)
    return x

In [33]:
# 定義人臉關鍵點檢測網路
img_input = Input(shape=imgs_train.shape)
x=inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
x=inception_resnet_block(x, 0.1, 'Incpetion_Block-B', activation='relu')
x=inception_resnet_block(x, 0.1, 'Incpetion_Block-C', activation='relu')

# 定義神經網路的輸入, hidden layer 以及輸出
x = GlobalAveragePooling2D()(x)
predictions = Dense(output_dim=30,activation='softmax')(x)

inputs = img_input
model = Model(inputs=img_input, name='inception_resnet', outputs=predictions)

# 配置 loss funtion 和 optimizer
model.compile(loss='mean_squared_error', optimizer='sgd')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=30)`
  
